<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/sensor_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import shutil
from scipy import signal


In [3]:
b = pd.read_csv('/content/drive/MyDrive/waypoint/5da138274db8ce0c98bbd3d2_waypoint_train.csv')
b_ = b[b['path_id'] == '5dc65d4c1cda3700060305cc']

In [4]:
timestamp_list = list(b_['timestamp'].values)

In [25]:
timestamp_05_list = []
timestamp_05_list.append(timestamp_list[0])
for n in range(len(timestamp_list)):
    if n == len(timestamp_list) - 1:
        break
    timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
    timestamp_05_list.append(timestamp_05)
timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])

In [26]:
a

[1573278734679, 1573278738306, 1573278746359, 1573278750784]

In [12]:
timestamp_list

[1573278734679, 1573278741934, 1573278750784]

In [125]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [4]:
used_buildings[2]

'5d27075f03f801723c2e360f'

In [193]:
for building in used_buildings:
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_train.csv')
    used_path = sorted(waypoint_df['path_id'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                 np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/acc_50/z')

In [194]:
for building in used_buildings:
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_train.csv')
    used_path = sorted(waypoint_df['path_id'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                 np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/gyro_50/z')

In [195]:
for building in used_buildings:
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_train.csv')
    used_path = sorted(waypoint_df['path_id'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                 np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/mag_50/z')

In [196]:
for building in used_buildings:
    waypoint_df = pd.read_csv(f'/content/drive/MyDrive/waypoint/{building}_waypoint_train.csv')
    acc_df = pd.read_csv(f'/content/drive/MyDrive/rotation_vector/{building}_rotation_vector_train.csv')
    used_path = sorted(waypoint_df['path_id'].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        waypoint_path_df = waypoint_df[waypoint_df['path_id'] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(waypoint_path_df['timestamp'].values)
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])]
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                 np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/rotation_50/z')

In [197]:

for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/accelerometer/{building}_accelerometer_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/acc_50/z')

In [198]:
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/gyro_50/z')

In [199]:
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/magnetic_field/{building}_magnetic_field_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/mag_50/z')

In [200]:
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/rotation_vector/{building}_rotation_vector_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/rotation_50/z')

In [ ]:
for building in used_buildings:
    test_df = ssubm_df[ssubm_df[0] == building]
    acc_df = pd.read_csv(f'/content/drive/MyDrive/gyroscope/{building}_gyroscope_test.csv')
    used_path = sorted(test_df[1].value_counts().index.tolist())
    dfs = []
    for path in used_path:
        test_path_df = test_df[test_df[1] == path]
        acc_path_df = acc_df[acc_df['path_id'] == path]
        timestamp_list = list(test_path_df[2].values)
        timestamp_list = list(map(int, timestamp_list))
        timestamp_05_list = []
        timestamp_05_list.append(timestamp_list[0])
        dfs_ = []
        for n in range(len(timestamp_list)):
            if n == len(timestamp_list) - 1:
                break
            timestamp_05 = int((timestamp_list[n] + timestamp_list[n+1]) * 0.5)
            timestamp_05_list.append(timestamp_05)
        timestamp_05_list.append(timestamp_list[len(timestamp_list)-1])
        for n in range(len(timestamp_05_list)):
            if n == len(timestamp_list):
                break
            acc_path_timestamp_df = acc_path_df[(acc_path_df['timestamp'] >= timestamp_05_list[n]) & (acc_path_df['timestamp'] <= timestamp_05_list[n+1])].fillna(0)
            if len(acc_path_timestamp_df['z'].values) == 0:
                break
            acc_path_timestamp_list = acc_path_timestamp_df['z'].values
            acc_path_timestamp_list_ = np.interp(np.arange(0, len(acc_path_timestamp_list), len(acc_path_timestamp_list) * 0.0201),
                                                np.arange(0, len(acc_path_timestamp_list)), acc_path_timestamp_list)
            acc_path_timestamp_df_ = pd.DataFrame(acc_path_timestamp_list_).T
            acc_path_timestamp_df_.columns = [f'acc_z_{str(i)}' for i in range(50)]
            acc_path_timestamp_df_['timestamp'] = timestamp_list[n]
            acc_path_timestamp_df_['path_id'] = path
            dfs_.append(acc_path_timestamp_df_)
        if len(dfs_) > 0:
            df_ = pd.concat(dfs_)
        else:
            del dfs_
        dfs.append(df_)
    df = pd.concat(dfs)
    df = df[~df.duplicated()]
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/gyro_50/z')

In [202]:
a = pd.read_csv('/content/drive/MyDrive/acc_50/z/5a0546857ecc773753327266_test.csv')
a

,Unnamed: 0,acc_z_0,acc_z_1,acc_z_2,acc_z_3,acc_z_4,acc_z_5,acc_z_6,acc_z_7,acc_z_8,acc_z_9,acc_z_10,acc_z_11,acc_z_12,acc_z_13,acc_z_14,acc_z_15,acc_z_16,acc_z_17,acc_z_18,acc_z_19,acc_z_20,acc_z_21,acc_z_22,acc_z_23,acc_z_24,acc_z_25,acc_z_26,acc_z_27,acc_z_28,acc_z_29,acc_z_30,acc_z_31,acc_z_32,acc_z_33,acc_z_34,acc_z_35,acc_z_36,acc_z_37,acc_z_38,acc_z_39,acc_z_40,acc_z_41,acc_z_42,acc_z_43,acc_z_44,acc_z_45,acc_z_46,acc_z_47,acc_z_48,acc_z_49,timestamp,path_id
0,0,7.810059,7.541689,7.645388,7.224835,10.644713,9.413268,12.923731,11.657932,6.233214,7.147763,7.376616,11.079867,10.908597,10.680936,4.044207,7.277979,5.290065,11.709501,10.940061,11.811760,8.109066,6.163696,6.718818,9.740833,10.770959,9.624454,8.261315,6.520266,5.380174,9.305943,12.452858,13.888151,6.820219,5.422708,4.827523,10.705160,14.202105,13.248886,4.947925,9.114405,4.397699,9.516273,15.071572,13.377827,8.996966,6.611612,6.182982,11.295513,11.613634,10.076444,9,046cfa46be49fc10834815c6
1,0,11.867035,7.549535,10.308873,10.385222,7.478750,4.331392,12.304123,9.809507,7.172683,12.591861,10.762684,6.573620,6.740869,12.095978,9.912962,6.607064,10.417787,9.094691,6.683503,7.160697,8.897496,11.681865,8.363695,5.188910,7.307394,9.940822,6.553635,8.417054,9.366301,8.645178,9.103533,9.054840,8.711533,8.197525,9.136777,9.065550,9.117963,9.835247,9.512802,9.182887,11.857296,5.861889,6.853645,11.967425,11.795654,6.537374,5.207795,12.716055,10.534710,7.888782,9017,046cfa46be49fc10834815c6
2,0,5.631348,11.211360,12.927502,11.751061,5.941952,5.665984,12.016386,11.549196,13.298562,9.406809,4.812763,9.237690,11.799189,13.399942,6.683349,6.467162,9.564263,13.381159,13.532045,9.967247,5.431516,7.829106,8.678550,9.490656,8.041352,6.167156,6.044619,9.582377,10.561088,11.065384,5.926175,7.086564,6.684233,10.081635,10.240956,9.957057,6.268576,6.037181,6.853735,10.789165,11.646519,13.100600,7.135197,5.723191,11.366386,11.402125,13.280079,5.682346,6.833350,7.052292,15326,046cfa46be49fc10834815c6
3,0,10.250351,11.849565,10.541319,13.159689,6.936376,6.409390,3.392424,7.648950,9.358439,10.481340,8.488606,7.792973,5.741957,5.588080,5.129477,6.443135,8.786764,9.830387,9.187851,12.178194,7.248431,7.501096,8.373688,7.343206,10.120655,8.258746,11.857856,9.297411,9.342058,5.894874,8.817481,5.959247,7.780225,10.235438,11.535377,10.826825,14.381439,5.116834,8.325827,6.113099,6.583049,10.946913,10.526058,11.340219,13.829899,4.715027,9.760714,5.982929,6.104727,10.275502,18763,046cfa46be49fc10834815c6
4,0,11.048828,9.826854,5.981275,6.602743,7.717269,9.788683,10.357691,6.628530,4.846596,5.471032,9.677081,9.383824,8.737379,8.606937,7.671455,9.025771,11.085781,11.270266,6.574560,9.015542,8.356841,8.926154,9.759789,10.223579,11.548387,6.447164,8.371133,8.890038,10.076356,9.880516,11.013833,6.263001,7.928078,8.661240,11.483011,9.730363,7.082168,5.948537,7.398724,11.696277,12.307584,8.901543,7.410522,8.108214,10.918081,11.236831,5.686118,9.101534,6.286380,9.754194,22328,046cfa46be49fc10834815c6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0,12.773239,14.069470,8.356834,8.193386,12.066083,9.732135,6.077121,7.892767,12.965549,9.820589,7.904464,7.412297,10.474953,10.493946,6.280431,7.173586,9.859212,10.790998,7.889810,5.613211,8.041637,8.739522,9.148409,6.998724,9.189974,10.855323,8.972972,9.789340,9.283764,7.937217,9.113951,8.084397,8.360829,9.592411,7.963242,10.400173,9.350315,6.075425,8.262284,10.644777,12.076910,5.973921,5.245449,9.569993,14.070469,11.072300,7.006093,9.299447,11.607684,12.110060,35117,ffcd9524c80c0fa5bb859eaf
288,0,7.265381,8.670997,12.845375,10.859059,6.928023,9.823734,11.664595,9.407490,6.851798,7.724639,7.318422,8.639025,7.943309,6.280150,6.661333,5.636273,4.661176,9.700862,7.453840,8.093987,8.612253,8.555813,8.440253,8.420356,9.470171,9.532576,10.307218,9.801365,8.845944,9.469656,9.051394,9.420460,8.664741,9

In [177]:
df_

,acc_z_0,acc_z_1,acc_z_2,acc_z_3,acc_z_4,acc_z_5,acc_z_6,acc_z_7,acc_z_8,acc_z_9,acc_z_10,acc_z_11,acc_z_12,acc_z_13,acc_z_14,acc_z_15,acc_z_16,acc_z_17,acc_z_18,acc_z_19,acc_z_20,acc_z_21,acc_z_22,acc_z_23,acc_z_24,acc_z_25,acc_z_26,acc_z_27,acc_z_28,acc_z_29,acc_z_30,acc_z_31,acc_z_32,acc_z_33,acc_z_34,acc_z_35,acc_z_36,acc_z_37,acc_z_38,acc_z_39,acc_z_40,acc_z_41,acc_z_42,acc_z_43,acc_z_44,acc_z_45,acc_z_46,acc_z_47,acc_z_48,acc_z_49,timestamp,path_id
0,7.286331,8.315744,8.863499,8.404492,8.607544,8.727712,8.893501,8.445191,9.022031,9.466164,11.176825,11.656771,9.559497,6.363728,7.787437,6.609492,8.665828,11.989340,11.014598,12.607866,5.971828,6.334010,6.339126,11.313611,13.135671,12.340120,5.604832,10.058335,4.826508,5.094824,15.749318,9.685567,5.181431,11.953781,6.491610,8.229959,11.736264,10.137433,10.534497,7.343869,7.962341,5.976862,8.585343,10.399714,14.062765,5.423508,9.013651,5.616667,9.758642,11.339050,7,ffcd9524c80c0fa5bb859eaf
0,8.607941,7.785822,7.948477,11.965203,7.346760,6.396521,9.017500,10.813959,7.104161,7.869293,10.245766,7.754701,7.126026,8.061378,6.467966,4.897580,7.785479,11.242402,9.286881,10.609081,9.340824,9.285204,8.859496,8.530342,8.415627,8.975220,8.997306,9.166927,8.923801,8.932173,9.818448,9.952812,9.177309,9.333511,8.560795,10.437533,9.086269,11.426872,8.742503,9.226112,9.487911,9.350474,9.375034,9.480002,9.274753,9.908176,9.188514,9.182121,9.978887,8.465141,9294,ffcd9524c80c0fa5bb859eaf
0,9.335175,11.011503,12.266648,6.911242,11.128687,15.610844,7.332821,11.882651,11.674921,9.471231,9.599367,10.212145,8.360086,8.361898,13.604319,6.147809,6.487184,13.490888,8.037288,5.496495,9.220477,11.399540,5.536288,7.773108,7.685453,8.133417,8.232264,9.553094,10.560363,7.516420,9.398828,8.290532,8.113230,8.345551,7.980816,8.932355,11.278743,8.974014,7.792512,9.621340,12.305033,6.613197,5.740757,12.075491,10.233952,6.624783,11.603151,14.941656,6.025915,10.563841,19971,ffcd9524c80c0fa5bb859eaf
0,12.883392,7.665230,5.178652,10.494359,12.049450,7.345052,5.273296,9.528276,12.369705,8.895792,6.709563,10.591816,12.244854,6.927209,8.996814,12.609353,11.919440,7.754854,8.975241,12.174464,9.698196,7.584466,8.533206,9.042795,9.241730,8.508475,9.062494,8.538738,9.099690,9.410120,9.958043,9.866075,12.686562,6.893274,6.662272,11.009888,12.720007,8.204065,6.218867,11.697008,10.802045,7.354998,10.312500,12.714594,7.555489,4.265638,10.254487,14.613665,8.628178,9.420128,26997,ffcd9524c80c0fa5bb859eaf
0,12.773239,14.069470,8.356834,8.193386,12.066083,9.732135,6.077121,7.892767,12.965549,9.820589,7.904464,7.412297,10.474953,10.493946,6.280431,7.173586,9.859212,10.790998,7.889810,5.613211,8.041637,8.739522,9.148409,6.998724,9.189974,10.855323,8.972972,9.789340,9.283764,7.937217,9.113951,8.084397,8.360829,9.592411,7.963242,10.400173,9.350315,6.075425,8.262284,10.644777,12.076910,5.973921,5.245449,9.569993,14.070469,11.072300,7.006093,9.299447,11.607684,12.110060,35117,ffcd9524c80c0fa5bb859eaf
0,7.265381,8.670997,12.845375,10.859059,6.928023,9.823734,11.664595,9.407490,6.851798,7.724639,7.318422,8.639025,7.943309,6.280150,6.661333,5.636273,4.661176,9.700862,7.453840,8.093987,8.612253,8.555813,8.440253,8.420356,9.470171,9.532576,10.307218,9.801365,8.845944,9.469656,9.051394,9.420460,8.664741,9.532178,9.448431,9.158292,9.159116,9.209829,9.516774,9.219374,9.542788,8.439070,10.320008,8.313909,11.965478,14.865151,7.915838,7.165566,11.676696,12.619706,41230,ffcd9524c80c0fa5bb859eaf
0,8.261978,10.424446,15.289512,5.946676,10.953120,6.813327,3.835667,12.434956,6.637432,6.421695,12.949893,10.198702,9.294928,10.844794,9.722742,9.404312,10.290533,9.368041,10.353137,7.844299,9.623275,7.456901,9.115575,7.980056,8.881096,10.856766,9.617316,8.785385,9.686565,9.255550,8.741948,9.284602,8.817713,9.978052,9.018919,8.751063,6.051198,8.300516,12.352728,7.337110,6.194808,13.701259,7.031176,7.569763,10.438357,7.221651,11.143262,13.228386,4.546979,11.741101,51634,ffcd9524c80c0fa5bb859eaf
0,15.130341,6.534542,11.052624,9.808242,7.179471,4.036868,13.2910

In [168]:
a[a['path_id'] == 'bb84ab5e77fc9f5fdbd52827']

,Unnamed: 0,acc_z_0,acc_z_1,acc_z_2,acc_z_3,acc_z_4,acc_z_5,acc_z_6,acc_z_7,acc_z_8,acc_z_9,acc_z_10,acc_z_11,acc_z_12,acc_z_13,acc_z_14,acc_z_15,acc_z_16,acc_z_17,acc_z_18,acc_z_19,acc_z_20,acc_z_21,acc_z_22,acc_z_23,acc_z_24,acc_z_25,acc_z_26,acc_z_27,acc_z_28,acc_z_29,acc_z_30,acc_z_31,acc_z_32,acc_z_33,acc_z_34,acc_z_35,acc_z_36,acc_z_37,acc_z_38,acc_z_39,acc_z_40,acc_z_41,acc_z_42,acc_z_43,acc_z_44,acc_z_45,acc_z_46,acc_z_47,acc_z_48,acc_z_49,timestamp,path_id
181,0,8.188354,7.863768,7.925426,8.286307,8.275124,8.381409,8.296481,8.410122,8.473882,8.686764,8.595352,8.560999,8.334111,7.193798,9.615039,10.266028,7.818299,7.006624,7.613415,8.816443,9.372447,9.198599,9.227123,9.806665,9.969594,11.356533,11.125171,9.744507,7.227696,5.630582,7.262719,6.598408,6.728031,7.901422,9.066752,10.371926,9.989556,10.277334,11.034870,5.017755,6.835789,5.983137,6.153338,6.642576,10.492125,9.490927,10.518039,9.350487,12.173168,6.348198,8,bb84ab5e77fc9f5fdbd52827
182,0,6.258621,6.601618,8.726176,10.704203,5.794310,6.636036,8.426292,9.858506,7.299250,7.752337,8.948289,7.546497,6.504832,8.062550,8.695419,5.738493,7.990704,7.243683,9.439145,10.425555,9.579429,8.558426,8.210629,7.063175,5.604501,7.526284,10.746301,11.442403,6.393318,7.122278,9.691585,8.913554,5.885443,7.359461,10.041392,6.641268,5.967962,11.014238,10.943099,5.880820,6.736453,10.120573,9.573794,6.419097,9.812798,8.759194,5.574068,7.909375,9.420172,8.770807,6034,bb84ab5e77fc9f5fdbd52827
183,0,5.049545,5.379882,10.018741,6.548062,6.018769,9.359126,7.768480,7.185668,9.495925,9.641019,6.163226,7.237159,9.415367,4.520555,4.924820,9.401704,7.573384,6.337097,7.907017,7.651379,8.093278,7.833548,7.025261,4.479880,7.395949,7.344594,6.893625,6.804487,7.443449,10.145419,8.932031,7.555225,7.962995,9.074131,9.425788,7.526461,8.529120,8.310756,10.813348,5.942452,10.562661,9.405468,11.419868,6.610866,7.449843,7.140242,5.991157,6.492244,7.796595,9.260405,14821,bb84ab5e77fc9f5fdbd52827
184,0,6.378937,8.318100,9.631556,8.427001,5.845614,8.307579,8.311771,5.946264,5.636665,8.911104,7.485302,5.240107,7.235914,9.967782,6.149824,7.965059,8.396930,10.018262,8.224058,4.964040,6.375249,5.253560,6.418446,6.973164,5.770651,7.309827,5.881587,6.510832,8.414971,7.310398,8.200986,8.690878,7.264209,6.675837,6.587998,6.802118,6.894604,5.965146,8.477955,8.225848,8.365903,8.068754,7.929030,9.033288,7.830739,6.986234,8.442707,9.972920,6.289614,5.927135,23631,bb84ab5e77fc9f5fdbd52827
185,0,11.784439,10.966147,5.024783,7.186952,9.798364,5.585164,6.016003,8.968279,10.906274,8.429768,9.888640,9.591067,7.089788,6.740589,7.688728,7.067941,6.402272,6.362135,7.298820,6.566063,5.866210,8.197251,8.700167,9.031017,9.623609,7.728902,8.628626,7.254342,7.877229,8.420478,8.825779,8.721494,6.723247,8.575938,9.680472,10.552332,7.555418,8.680405,11.148912,7.075492,6.016243,9.403174,10.223993,7.189658,10.684579,11.119775,7.726551,7.752973,10.652662,7.273266,30734,bb84ab5e77fc9f5fdbd52827
186,0,7.492233,10.320756,10.752105,6.770578,7.176003,9.689043,8.399494,6.827321,9.408923,10.153331,6.896357,6.851841,10.576606,10.062088,8.742748,9.717082,9.198248,5.857823,7.239050,8.379222,8.848762,6.116939,8.160507,9.095442,9.728216,7.579944,8.090873,6.627609,8.228140,8.988412,7.961481,9.739985,8.586045,7.856115,7.383663,9.452708,10.254811,5.858765,8.276922,10.013756,12.866318,7.803065,8.041065,11.402471,5.721658,6.346243,9.415362,11.845342,7.673158,8.161553,39714,bb84ab5e77fc9f5fdbd52827
187,0,11.019501,6.549424,9.004897,11.554414,5.460832,12.578748,6.274239,10.203109,6.795062,8.599420,9.677989,5.872500,9.046305,9.425140,6.063708,9.657228,6.159421,6.538110,7.635512,9.137551,6.999318,7.614399,9.487620,9.306399,9.090683,9.021115,9.023206,8.565252,8.334447,7.256324,6.005634,8.452488,9.152274,6.663688,11.376879,5.739613,8.917752,13.184994,5.152086,12.175640,5.154125,11.078279,12.769423,5.633285,12.317822,6.118705,8.525688,7.610845,7.337010,8.622603,47119,bb84ab5e77fc9f5fdbd52827
188,0,8.728836,7.064271,9.499113,6.413271,8.935090,8.734553,7.426721,7.757

In [175]:
acc_df[acc_df['path_id'] == 'bb84ab5e77fc9f5fdbd52827']

,Unnamed: 0,timestamp,x,y,z,accuracy,path_id
92859,0,131,-1.053711,5.457107,8.188354,2.0,bb84ab5e77fc9f5fdbd52827
92860,1,151,-1.029755,5.405045,8.012970,2.0,bb84ab5e77fc9f5fdbd52827
92861,2,171,-0.965714,5.282333,7.923202,2.0,bb84ab5e77fc9f5fdbd52827
92862,3,191,-0.859772,5.358353,7.856751,2.0,bb84ab5e77fc9f5fdbd52827
92863,4,211,-0.801712,5.318848,7.791519,2.0,bb84ab5e77fc9f5fdbd52827
...,...,...,...,...,...,...,...
97075,4216,85182,-0.952545,7.710052,5.777985,2.0,bb84ab5e77fc9f5fdbd52827
97076,4217,85202,-1.157257,7.774704,5.284790,2.0,bb84ab5e77fc9f5fdbd52827
97077,4218,85222,-1.582230,8.040466,4.221756,2.0,bb84ab5e77fc9f5fdbd52827
97078,4219,85242,-1.247635,7.779480,5.860580,2.0,bb84ab5e77fc9f5fdbd52827


In [180]:
for path in used_path:
    print(path)
    print(a[a['path_id'] == path].shape)
    print(test_df[test_df[1] == path].shape)
    print('--------------------------------------')


046cfa46be49fc10834815c6
(9, 53)
(9, 3)
--------------------------------------
05d052dde78384b0c543d89c
(8, 53)
(8, 3)
--------------------------------------
0c06cc9f21d172618d74c6c8
(14, 53)
(14, 3)
--------------------------------------
146035943a1482883ed98570
(10, 53)
(10, 3)
--------------------------------------
1ef2771dfea25d508142ba06
(8, 53)
(8, 3)
--------------------------------------
3506b3b626f494b0f0b934ca
(12, 53)
(12, 3)
--------------------------------------
3e0aebb66ef39150bbc27c24
(11, 53)
(11, 3)
--------------------------------------
3e1d46017fbfcc8136bd1e9b
(12, 53)
(12, 3)
--------------------------------------
412da1891c4780f6f0f7f4bc
(10, 53)
(10, 3)
--------------------------------------
6d89334316127640cff99800
(16, 53)
(16, 3)
--------------------------------------
72963a8c7eb520c56f88a536
(12, 53)
(12, 3)
--------------------------------------
7d2d723a30cce824aad9915b
(10, 53)
(10, 3)
--------------------------------------
947e17f82dbddfbdb4cb2447
(11, 53)


In [132]:
len(test_df[2].unique())

278

In [117]:
b = pd.read_csv('/content/drive/MyDrive/acc_50/z/5a0546857ecc773753327266_train.csv')
b

,Unnamed: 0,acc_z_0,acc_z_1,acc_z_2,acc_z_3,acc_z_4,acc_z_5,acc_z_6,acc_z_7,acc_z_8,acc_z_9,acc_z_10,acc_z_11,acc_z_12,acc_z_13,acc_z_14,acc_z_15,acc_z_16,acc_z_17,acc_z_18,acc_z_19,acc_z_20,acc_z_21,acc_z_22,acc_z_23,acc_z_24,acc_z_25,acc_z_26,acc_z_27,acc_z_28,acc_z_29,acc_z_30,acc_z_31,acc_z_32,acc_z_33,acc_z_34,acc_z_35,acc_z_36,acc_z_37,acc_z_38,acc_z_39,acc_z_40,acc_z_41,acc_z_42,acc_z_43,acc_z_44,acc_z_45,acc_z_46,acc_z_47,acc_z_48,acc_z_49,timestamp,path_id
0,0,8.115402,9.083643,10.394832,9.737289,10.099504,9.570830,11.127541,8.524143,9.533763,8.156872,9.718386,9.768703,13.171182,9.657311,5.784263,8.799868,10.732313,10.245083,14.894110,8.275374,7.626399,8.831167,11.192212,11.183156,13.072468,8.515871,9.178401,10.025433,10.177518,12.823070,9.280182,6.792580,8.064178,9.945452,11.434900,12.793089,9.373636,6.901324,9.321726,8.365552,12.574969,10.649690,8.399703,7.094332,10.132006,9.241303,11.687890,6.615379,7.454736,8.710425,1561368762838,5d10a1669c50c70008fe8977
1,0,9.102417,8.965151,10.402376,8.337900,8.242279,9.127050,10.420593,6.325279,10.634862,7.533216,11.256797,8.294593,12.504621,8.251397,10.444956,8.096568,10.291451,9.521030,10.162962,9.137966,9.830050,9.040053,9.815567,10.087891,8.709008,8.254384,9.188135,8.847558,9.943502,6.729712,9.780512,8.731337,10.296287,8.618206,11.297883,7.501352,11.340792,9.502830,9.451991,9.185677,5.627467,9.986349,8.094294,11.291610,7.327456,14.389585,8.705078,14.740538,9.570683,9.107801,1561368774057,5d10a1669c50c70008fe8977
2,0,9.994858,13.677186,9.161700,13.639425,6.756186,11.690905,8.480951,9.919421,9.523327,10.068057,6.136718,8.578520,10.562289,8.460993,11.610712,8.088047,11.182832,7.095168,10.077237,7.262808,9.275472,8.328358,9.642932,14.497984,8.520471,12.550524,7.315482,10.867154,9.351291,9.427042,10.075728,9.318975,13.218117,8.921707,11.706323,8.069509,8.964759,7.428878,9.236748,8.943101,9.460341,9.256253,9.478644,9.603351,8.724142,9.984957,5.759421,8.973121,13.780974,6.669017,1561368793535,5d10a1669c50c70008fe8977
3,0,10.339630,13.462623,7.093830,10.739798,10.267708,9.503172,11.228691,9.632910,9.247168,13.849450,9.777395,9.599059,9.628868,9.225099,8.114027,9.662890,9.794302,9.214974,8.754701,9.134763,11.223694,10.050836,9.796724,9.312655,10.056784,10.665914,11.290793,8.681313,9.356116,13.365043,7.542807,10.619844,6.465407,9.047176,14.287344,7.068551,10.081395,8.875184,8.073752,12.763820,8.018412,10.920891,12.296147,8.311986,11.612484,8.896961,9.852821,11.562358,7.677847,11.186318,1561368798851,5d10a1669c50c70008fe8977
4,0,8.377579,7.353938,12.702103,6.733201,11.668407,7.767615,9.932011,7.248667,8.807633,9.323590,8.951136,12.538849,8.526012,12.853725,6.662424,11.355776,9.761414,9.723979,8.445658,9.173351,11.826576,9.275883,12.571917,8.789494,9.535469,8.822151,8.259207,11.927841,7.769600,9.330502,11.210667,10.138022,11.157837,8.253972,8.405556,9.668987,9.160331,8.994004,9.547894,8.942903,9.460953,9.791270,9.043682,9.454668,9.590237,10.766396,10.003015,9.325607,9.937988,9.371956,1561368814782,5d10a1669c50c70008fe8977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,0,11.285248,11.337743,10.637701,7.483926,7.658104,11.119141,10.220927,8.315590,6.257018,8.375941,8.984350,8.926907,5.319161,5.537246,9.553972,6.216062,6.323613,5.765098,7.049015,9.363143,8.609720,7.262259,6.163510,9.993279,9.116207,8.507902,9.106367,7.954983,6.828363,8.254407,10.627223,9.772228,5.727102,6.364469,9.572239,10.466478,10.411731,7.948535,6.230102,10.192593,10.304401,8.502013,4.877365,8.699434,12.319186,12.249217,6.224295,5.535987,10.572204,10.213478,1578483574917,5e15bf91f4c3420006d52341
2478,0,12.360245,6.726361,6.276723,6.893469,6.159369,6.107866,9.252229,10.354180,12.078674,10.712773,10.953941,9.101232,4.193681,5.184819,6.705265,6.617733,8.672087,10.159800,8.727302,10.965849,5.967551,8.257628,6.751502,5.034423,7.610651,6.838938,4.969509

In [122]:
len(a['timestamp'].unique())

278

In [136]:
acc_df

,Unnamed: 0,timestamp,x,y,z,accuracy,path_id
0,0,136,-0.574860,5.930573,7.481476,2.0,146035943a1482883ed98570
1,1,156,-0.799911,5.992218,7.241455,2.0,146035943a1482883ed98570
2,2,176,-0.826248,5.987427,7.306686,2.0,146035943a1482883ed98570
3,3,196,-0.743057,5.988632,7.240250,2.0,146035943a1482883ed98570
4,4,216,-0.753220,6.140060,7.322250,2.0,146035943a1482883ed98570
...,...,...,...,...,...,...,...
111716,5198,104987,-1.777344,5.369125,8.298492,2.0,947e17f82dbddfbdb4cb2447
111717,5199,105007,-1.701935,5.406830,6.622543,2.0,947e17f82dbddfbdb4cb2447
111718,5200,105027,-1.915008,5.109360,6.538742,2.0,947e17f82dbddfbdb4cb2447
111719,5201,105047,-1.936569,5.200333,8.996994,2.0,947e17f82dbddfbdb4cb2447


In [141]:
acc_df['timestamp'].sum()

4749106680

In [143]:
acc_df['timestamp'].unique().sum()

3070949554